In [ ]:
mkdir -p training_datasets

wget -P training_datasets https://zenodo.org/records/13831403/files/ids_train_casp12nr50_nr70Ig_nr40Others.fasta

wget -P training_datasets https://zenodo.org/records/13831403/files/sidechainnet_casp12_50.pkl

wget -P training_datasets https://zenodo.org/records/13831403/files/AbSCSAbDAb_trainnr90_bkandcbcoords_aug2022.h5
wget -P training_datasets https://zenodo.org/records/13831403/files/ppi_trainset_5032_noabag_aug2022.h5
wget -P training_datasets https://zenodo.org/records/13831403/files/train_af_paired_nr70.h5

# Is this the test set?
wget -P training_datasets https://zenodo.org/records/13831403/files/testset_rabd-dms-vhh_backboneandcb_oct2022.h5

In [ ]:
### COPY SIDECHAINNET TO LOCAL TEMPORARY DIRECTORY

sidechainnet = './training_datasets/sidechainnet_casp12_50.pkl'
sidechainnet_temp = '/tmp/sidechainnet_casp12_50.pkl'

if not os.path.exists(sidechainnet_temp):
    shutil.copy(sidechainnet, sidechainnet_temp)

In [ ]:
### MODEL AND TRAINING PARAMETERS
LAYERS=4
HEADS=8
DIM=256
OLD=0
BS=1
SS=50 # protein model was trained on 90ss
save_every=5
gmodel=egnn-trans-ma
atom_types=backbone_and_cb
NN=48

### Training seed and output directory
SEED=1
MODELS_DIR=models_out_dir_seed_$SEED
EPOCHS=10

### WANDB ENTITY
WANDB_ENTITY="fadh-johns-hopkins-university"


In [ ]:
gd2_dataset_ids = os.path.join(os.getcwd(), 'training_datasets', 'ids_train_casp12nr50_nr70Ig_nr40Others.fasta')

In [ ]:
### Get SLURM variables and list of gpus

slurm_ntasks = int(os.getenv('SLURM_NTASKS', 1))  # Default to 1 if not set
slurm_job_num_nodes = int(os.getenv('SLURM_JOB_NUM_NODES', 1))  # Default to 1 if not set

# Calculate n_procs
n_procs = slurm_ntasks // slurm_job_num_nodes

# Default value for num_gpus
num_gpus = 0

slurm_step_gpus = os.getenv('SLURM_STEP_GPUS')
gpus_list = slurm_step_gpus.split(',')
num_gpus = len(gpus_list)


In [ ]:
# Run training using shell execution in Jupyter
!python3 ./MaskedProteinEnT/train_masked_model.py \
  --save_every {save_every} --lr 0.00001 --batch_size {BS} \
  --heads {HEADS} --model_dim {DIM} --epochs {EPOCHS} --dropout 0.2 \
  --masking_rate_max 0.25 --topk_metrics 1 --layers {LAYERS} \
  --num_gpus {num_gpus} --crop_sequences \
  --scn_sequence_similarity {SS} --protein_gmodel {gmodel} \
  --lr_patience 350 --lr_cooldown 20 --max_ag_neighbors {NN} \
  --atom_types {atom_types} \
  --file_with_selected_scn_ids_for_training {gd2_dataset_ids} \
  --lightning_save_last_model --use_scn --num_procs {n_procs} \
  --output_dir {MODELS_DIR} --seed {SEED} --wandb_entity {WANDB_ENTITY}
